In [2]:
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [26]:
hltb = requests.get('https://howlongtobeat.com/game.php?id=4250').content
hltb_soup = BeautifulSoup(hltb, "lxml")

def get_game_title(soup):
    return soup.find('div', attrs = {'class': 'profile_header shadow_text'}).get_text().strip('\n').strip(' $')

get_game_title(hltb_soup)

'Half-Life 2: Episode One'

In [4]:
## Need to generate numbers to find all titles, including 4 and 5 digits

In [5]:
def text_from_tag(tag):
    return tag.get_text().strip('\n').strip(' $')

In [12]:
def get_average_time(times):
    return text_from_tag(times.findAll('td')[2])

def get_median_time(times):
    return text_from_tag(times.findAll('td')[3])

def get_rushed_time(times):
    return text_from_tag(times.findAll('td')[4])

def get_leisure_time(times):
    return text_from_tag(times.findAll('td')[5])

def get_poll_count(times):
    return text_from_tag(times.findAll('td')[1])

def get_play_title(times):
    return text_from_tag(times.findAll('td')[0])


In [7]:
def get_main_story_times(soup):
    return soup.findAll('tr')[1]

def get_main_story_extras_times(soup):
    return soup.findAll('tr')[2]

def get_completionist_times(soup):
    return soup.findAll('tr')[3]

def get_all_playstyles_times(soup):
    return soup.findAll('tr')[4]

get_main_story_times(hltb_soup)

<tr>
<td>Main Story</td>
<td class="center time_100_text">791 </td> <td>3h 55m </td>
<td>4h </td>
<td>2h 40m </td>
<td>6h 58m </td>
</tr>

In [44]:
def generate_row(tag, soup):
    return pd.DataFrame(data = [[get_game_title(soup), get_play_title(tag), get_poll_count(tag), 
                                 get_average_time(tag), get_median_time(tag), get_rushed_time(tag), 
                                 get_leisure_time(tag)]],
                       columns = ['Title', 'Playstyle', 'Polled', 'Average', 'Median', 'Rushed', 'Leisure'])

def cook_soup(id_no):
    hltb = requests.get(str('https://howlongtobeat.com/game.php?id=' + id_no)).content
    hltb_soup = BeautifulSoup(hltb, "lxml")
    return hltb_soup

def cook_soup(id_no):
    return BeautifulSoup(requests.get(str('https://howlongtobeat.com/game.php?id=' + id_no)).content, "lxml")

def create_game_entry(id_no):
    soup = cook_soup(id_no)
    main = generate_row(get_main_story_times(soup), soup)
    main_extra = generate_row(get_main_story_extras_times(soup), soup)
    completionist = generate_row(get_completionist_times(soup), soup)
    all_playstyles = generate_row(get_all_playstyles_times(soup), soup)
    return main.append(main_extra).append(completionist).append(all_playstyles).set_index(['Title', 'Playstyle'])


half_life_episode_two = create_game_entry("4250")
half_life_episode_two

Polled Average Median  Rushed  Leisure
Title                    Playstyle                                            
Half-Life 2: Episode One Main Story        791  3h 55m     4h  2h 40m   6h 58m
                         Main + Extras     217  4h 30m     4h  3h 07m   7h 45m
                         Completionists    185  5h 32m     5h  3h 23m  18h 38m
                         All PlayStyles   1.2K  4h 16m     4h  2h 46m  18h 04m

In [45]:
# add game name as a further col? Then could be a single dataframe

In [46]:
rdr2 = create_game_entry('27100')
rdr2

Polled   Average    Median    Rushed  \
Title                 Playstyle                                             
Red Dead Redemption 2 Main Story        161   43h 34m   43h 57m   31h 06m   
                      Main + Extras     323   74h 15m       70h   51h 02m   
                      Completionists     36  139h 59m  141h 49m  106h 02m   
                      All PlayStyles    520   69h 18m       64h   42h 33m   

                                       Leisure  
Title                 Playstyle                 
Red Dead Redemption 2 Main Story       52h 23m  
                      Main + Extras   124h 03m  
                      Completionists  170h 57m  
                      All PlayStyles  133h 49m